In [121]:
import requests_html
from requests_html import HTMLSession
import  time
import nest_asyncio

In [122]:
root = "https://ipspeed.info"
url = "https://ipspeed.info/freevpn_openvpn.php?language=en"

In [123]:
nest_asyncio.apply()

s = HTMLSession()

def getPage(session, url):
    r = requests_html.HTMLResponse(session)

    for _ in range(5):
        try:
            r = s.get(url)
            print(f"Successfully connected to {url}")
            return r
        except:
            print(f"Trying to connect to {url}")
            time.sleep(1)
    
    print(f"Cannot reach page")
    return r

r = getPage(s, url)

r.text

Successfully connected to https://ipspeed.info/freevpn_openvpn.php?language=en


'<!DOCTYPE html>\r\n<html lang="en">\n    <head>\r\n        <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\r\n        <title>IPSpeed | OpenVPN Free and Secure</title>\r\n        <meta name="description" content="Connect to free OpenVPN for a secure and anonymous internet on IPSpeed!">\r\n        <meta name="keywords" content="free OpenVPN, VPN, anonymity, security, internet, IPSpeed">\r\n        <link rel="canonical" href="https://ipspeed.info/freevpn_openvpn.php?language=en">\r\n        <link rel="alternate" media="only screen and (max-width: 640px)" href="https://m.ipspeed.info/freevpn_openvpn.php?language=en">\r\n\t<link rel="alternate" hreflang="en" href="https://ipspeed.info/freevpn_openvpn.php?language=en">\r\n\t<link rel="alternate" hreflang="de" href="https://ipspeed.info/freevpn_openvpn.php?language=de">\r\n\t<link rel="alternate" hreflang="es" href="https://ipspeed.info/freevpn_openvpn.php?language=es">\r\n\t<link rel="alternate" hreflang="fr" href="https

In [124]:
from bs4 import BeautifulSoup

In [125]:
soup = BeautifulSoup(r.text, 'html.parser')
soup.body

<body>
<center>
<div style="width: 1200px;">
<a href="https://ipspeed.info"><img src="logo.webp" style="margin: 30px 0 15px; float: left;"/></a>
<a href="?language=tr"><div class="menu" style="float: right;">TR</div></a><a href="?language=ru"><div class="menu" style="float: right; margin-left: 15px; margin-right: 15px;">RU</div></a><a href="?language=fr"><div class="menu" style="float: right;">FR</div></a><a href="?language=es"><div class="menu" style="float: right; margin-left: 15px; margin-right: 15px;">ES</div></a><a href="?language=de"><div class="menu" style="float: right; margin-left: 15px;">DE</div></a><div class="menu_crt" style="float: right;">EN</div><div class="menu_" style="float: right; margin-left: 30px; margin-right: 30px;">•</div><a href="freevpn_sstp.php?language=en"><div class="menu" style="float: right;">SSTP</div></a><div class="menu_" style="float: right;"> | </div><a href="freevpn_l2tpipsec.php?language=en"><div class="menu" style="float: right;">L2TP/IPsec</div><

In [126]:
elements = soup.find('body').find_all(download="")

def prev_div(div):
    return div.previous_sibling.previous_sibling

def next_div(div):
    return div.next_sibling.next_sibling

def get_block_by_a(element):
    parent = element.parent
    return (prev_div(parent), parent, next_div(parent), next_div(next_div(parent)))

blocks = [get_block_by_a(x) for x in elements]
blocks

[(<div class="list" style="float: left; width: 293px;">Belarus</div>,
  <div class="list" style="float: left; width: 393px;"><a download="" href="https://ipspeed.info/ovpn/46.53.249.223_udp_42549.ovpn">46.53.249.223_udp_42549.ovpn</a></div>,
  <div class="list" style="float: left; width: 150px; text-align: center;">7 hours</div>,
  <div class="list" style="float: left; width: 100px; text-align: center;">11 ms</div>),
 (<div class="list" style="float: left; width: 293px;">Germany</div>,
  <div class="list" style="float: left; width: 393px;"><a download="" href="https://ipspeed.info/ovpn/91.11.72.201_udp_1194.ovpn">91.11.72.201_udp_1194.ovpn</a><br/><a download="" href="https://ipspeed.info/ovpn/91.11.72.201_tcp_1194.ovpn">91.11.72.201_tcp_1194.ovpn</a></div>,
  <div class="list" style="float: left; width: 150px; text-align: center;">2 days</div>,
  <div class="list" style="float: left; width: 100px; text-align: center;">11 ms</div>),
 (<div class="list" style="float: left; width: 293px;

In [127]:
str(blocks[0][0].contents[0])

'Belarus'

In [128]:
blocks[0][1].a.get('href')

'https://ipspeed.info/ovpn/46.53.249.223_udp_42549.ovpn'

In [129]:
uptime_str = str(blocks[0][2].contents[0])


def uptime_str_to_int(s: str):
    if (s == None):
        return 0

    split = s.split(' ')

    if (len(split) < 2):
        return 0

    num = int(s[0])
    if (split[1] == 'mins'):
        return num
    elif (split[1] == 'hours'):
        return 60*num
    elif (split[1] == 'days'):
        return 24*60*num
    return 0

uptime_str_to_int(uptime_str)

420

In [130]:
def delay_str_to_int(s: str):
    split = s.split(' ')

    if (len(split) < 2):
        return 0

    if (split[1] == 'ms'):
        return int(split[0])
    
    return 60


delay_str = str(blocks[0][3].contents[0])

delay_str_to_int(delay_str)


11

In [131]:
class Connection:
    def __init__(self, country, url, uptime, delay):
        self.country = country
        self.uptime = uptime
        self.delay = delay
        self.url = url

    def __repr__(self):
        return f'Connection({self.country}, {self.url}, {self.uptime}, {self.delay})'
        


def convert_block(block):
    country = str(block[0].contents[0])
    url = block[1].a.get('href')
    uptime = uptime_str_to_int(str(block[2].contents[0]))
    delay = delay_str_to_int(str(block[3].contents[0]))
    return Connection(country = country, url = url, uptime = uptime, delay = delay)

connections = [convert_block(x) for x in blocks]
connections

[Connection(Belarus, https://ipspeed.info/ovpn/46.53.249.223_udp_42549.ovpn, 420, 11),
 Connection(Germany, https://ipspeed.info/ovpn/91.11.72.201_udp_1194.ovpn, 2880, 11),
 Connection(Germany, https://ipspeed.info/ovpn/91.11.72.201_udp_1194.ovpn, 2880, 11),
 Connection(Germany, https://ipspeed.info/ovpn/95.91.195.145_udp_53819.ovpn, 1440, 13),
 Connection(Iran (ISLAMIC Republic Of), https://ipspeed.info/ovpn/80.210.32.162_udp_1195.ovpn, 180, 37),
 Connection(Japan, https://ipspeed.info/ovpn/180.5.47.168_udp_1699.ovpn, 0, 10),
 Connection(Japan, https://ipspeed.info/ovpn/180.5.47.168_udp_1699.ovpn, 0, 10),
 Connection(Japan, https://ipspeed.info/ovpn/126.217.201.240_udp_1706.ovpn, 2880, 4),
 Connection(Japan, https://ipspeed.info/ovpn/126.217.201.240_udp_1706.ovpn, 2880, 4),
 Connection(Japan, https://ipspeed.info/ovpn/122.208.194.129_udp_1194.ovpn, 4320, 4),
 Connection(Japan, https://ipspeed.info/ovpn/122.208.194.129_udp_1194.ovpn, 4320, 4),
 Connection(Japan, https://ipspeed.info/ov

In [132]:
good_connections = [x for x in connections if (x.country == 'Japan')]
good_connections.sort(key = lambda x: x.delay )
good_connections

[Connection(Japan, https://ipspeed.info/ovpn/106.179.239.207_udp_1297.ovpn, 0, 1),
 Connection(Japan, https://ipspeed.info/ovpn/106.179.239.207_udp_1297.ovpn, 0, 1),
 Connection(Japan, https://ipspeed.info/ovpn/39.110.119.41_udp_1940.ovpn, 2880, 2),
 Connection(Japan, https://ipspeed.info/ovpn/39.110.119.41_udp_1940.ovpn, 2880, 2),
 Connection(Japan, https://ipspeed.info/ovpn/116.82.5.94_udp_1684.ovpn, 1440, 3),
 Connection(Japan, https://ipspeed.info/ovpn/116.82.5.94_udp_1684.ovpn, 1440, 3),
 Connection(Japan, https://ipspeed.info/ovpn/130.62.37.153_udp_35355.ovpn, 1440, 3),
 Connection(Japan, https://ipspeed.info/ovpn/59.147.10.203_udp_1669.ovpn, 420, 3),
 Connection(Japan, https://ipspeed.info/ovpn/106.150.248.216_udp_1824.ovpn, 0, 3),
 Connection(Japan, https://ipspeed.info/ovpn/106.150.248.216_udp_1824.ovpn, 0, 3),
 Connection(Japan, https://ipspeed.info/ovpn/126.217.201.240_udp_1706.ovpn, 2880, 4),
 Connection(Japan, https://ipspeed.info/ovpn/126.217.201.240_udp_1706.ovpn, 2880, 

In [133]:
import os

os.system('mkdir -p configs')

0

In [ ]:
import requests

name = 'virtual-penis-network'
filename = f'{name}.ovpn'
path = f'configs/{filename}'

In [137]:
r = requests.get(good_connections[20].url)
open(path, 'wb').write(r.content)

os.system(f'nmcli connection delete {name}')
os.system(f'nmcli connection import type openvpn file {path}')
os.system(f'nmcli connection up {name}')

Подключение «virtual-penis-network» (92280af8-0ff2-41ce-af67-b2cd51366cbc) успешно удалено.
Подключение «virtual-penis-network» (901cd5e5-a858-4f5f-906b-e5f8c668d2c3) успешно добавлено.
Подключение успешно активировано (активный путь D-Bus: /org/freedesktop/NetworkManager/ActiveConnection/13)


0